In [9]:
# ✅ 0. 라이브러리 로드 및 시드 고정
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from itertools import product

torch.manual_seed(42)
np.random.seed(42)

In [10]:
#──────────────────────────────────────────────────────────────
# 1. 라이브러리
import pandas as pd, numpy as np, xgboost as xgb
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')
#──────────────────────────────────────────────────────────────
# 2. 데이터 로드 및 전처리 (관측 + 예측)
obs  = pd.read_csv('drive/MyDrive/데이터_분석과제_7_기상관측데이터_2401_2503.csv')
pred = pd.read_csv('drive/MyDrive/데이터_분석과제_7_기상예측데이터_2401_2503.csv')
obs.columns  = ['datetime', 'humidity_obs', 'temp_obs', 'pressure_obs']
pred.columns = ['datetime', 'solar_rad_pred', 'humidity_pred', 'absolute_humidity_pred', 'temp_pred', 'pressure_pred']

# 단위 통일 (pressure: mmHg → hPa)
obs['pressure_obs'] = obs['pressure_obs'] * 1.33322
obs['datetime']     = pd.to_datetime(obs['datetime'])
pred['datetime']    = pd.to_datetime(pred['datetime'])

df = pd.merge(obs, pred, on='datetime')

# 오차 변수 생성
df['temp_error']     = df['temp_pred']     - df['temp_obs']
df['humidity_error'] = df['humidity_pred'] - df['humidity_obs']
df['pressure_error'] = df['pressure_pred'] - df['pressure_obs']

# 시간 파생변수
df['hour']  = df['datetime'].dt.hour
df['month'] = df['datetime'].dt.month
df['day']   = df['datetime'].dt.day
df['year']  = df['datetime'].dt.year

df['hour_sin']  = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos']  = np.cos(2 * np.pi * df['hour'] / 24)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

#──────────────────────────────────────────────────────────────
# 3. 이상치 제거
def get_outlier_indices(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return series[(series < lower) | (series > upper)].index

out_idx = set(get_outlier_indices(df['pressure_error']))
out_idx |= set(get_outlier_indices(df['temp_error']))
out_idx |= set(get_outlier_indices(df['humidity_error']))

drop_outliers = df.drop(index=out_idx).reset_index(drop=True)
ori_data = df.copy()


# 데이터 분할
train_df = drop_outliers[
    (drop_outliers["datetime"] >= "2024-01-01") & (drop_outliers["datetime"] < "2025-01-01")
]
val_df = ori_data[
    (ori_data["datetime"] >= "2025-01-01") & (ori_data["datetime"] < "2025-03-01")
]
test_df = ori_data[
    (ori_data["datetime"] >= "2025-03-01") & (ori_data["datetime"] < "2025-04-01")
]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# 공통 입력 컬럼
input_cols = [
    "temp_pred", "humidity_pred", "pressure_pred",
    "absolute_humidity_pred", "solar_rad_pred",
    "hour_sin", "hour_cos", "month_sin", "month_cos",
    "month", "day", "hour",
]

# ✅ 공통 Dataset 정의
class ForecastDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ✅ 공통 모델 정의
class ForecastMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, output_dim)
        )
    def forward(self, x):
        return self.net(x)

In [12]:
# ✅ 손실 및 평가 함수
class RMSE_MAE_Loss(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, pred, target):
        rmse = torch.sqrt(torch.mean((pred - target) ** 2))
        mae = torch.mean(torch.abs(pred - target))
        return (rmse + mae) / 2

def evaluate_single_target(y_true, y_pred, scaler_y):
    y_true = scaler_y.inverse_transform(y_true)
    y_pred = scaler_y.inverse_transform(y_pred)
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    mae = np.mean(np.abs(y_true - y_pred))
    return (rmse + mae) / 2


In [13]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler,
                scaler_y=None, num_epochs=3000, patience=50, verbose=True):
    best_val_score = float("inf")
    best_model_state = None
    epochs_no_improve = 0

    train_losses = []
    val_scores = []
    lrs = []

    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss = 0
        for xb, yb in train_loader:
            pred = model(xb)
            loss = criterion(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        train_losses.append(avg_loss)

        # 검증
        model.eval()
        val_preds, val_trues = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                pred = model(xb)
                val_preds.append(pred.numpy())
                val_trues.append(yb.numpy())

        val_score = evaluate_single_target(np.vstack(val_trues), np.vstack(val_preds), scaler_y)
        val_scores.append(val_score)
        lrs.append(optimizer.param_groups[0]["lr"])

        scheduler.step(val_score)

        if verbose and (epoch % 10 == 0 or epoch == 1):
            print(f"[Epoch {epoch:04d}] Train Loss: {avg_loss:.6f} | Val sAError: {val_score:.6f}")

        if val_score < best_val_score:
            best_val_score = val_score
            best_model_state = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                if verbose:
                    print(f"\n🛑 Early stopping at epoch {epoch}")
                break

    return best_model_state, train_losses, val_scores, lrs


In [14]:
def run_forecast_pipeline(target_col, lr, sched_patience, factor,
                          hidden_dim=128, train_patience=50, batch_size=64):
    print(f"\n🚀 Target: {target_col}")

    # 1. 입력/출력 정의
    X_train = train_df[input_cols].copy()
    X_val   = val_df[input_cols].copy()
    X_test  = test_df[input_cols].copy()

    y_train = train_df[[target_col]].copy()
    y_val   = val_df[[target_col]].copy()
    y_test  = test_df[[target_col]].copy()

    # 2. 정규화
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train = scaler_X.fit_transform(X_train)
    X_val   = scaler_X.transform(X_val)
    X_test  = scaler_X.transform(X_test)

    y_train = scaler_y.fit_transform(y_train)
    y_val   = scaler_y.transform(y_val)
    y_test  = scaler_y.transform(y_test)

    # 3. Dataloader
    train_loader = DataLoader(ForecastDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(ForecastDataset(X_val, y_val), batch_size=batch_size)
    test_loader  = DataLoader(ForecastDataset(X_test, y_test), batch_size=batch_size)

    # 4. 모델 구성
    model = ForecastMLP(input_dim=len(input_cols), hidden_dim=hidden_dim, output_dim=1)
    criterion = RMSE_MAE_Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
        patience=sched_patience, factor=factor
    )

    # 5. 학습
    best_model_state, _, _, _ = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler,
                                   scaler_y=scaler_y, patience=train_patience)
    model.load_state_dict(best_model_state)
    model.eval()

    # 6. 예측
    test_preds = []
    with torch.no_grad():
        for xb, _ in test_loader:
            pred = model(xb)
            test_preds.append(pred.numpy())

    test_preds = np.vstack(test_preds)
    preds = scaler_y.inverse_transform(test_preds).flatten()
    trues = scaler_y.inverse_transform(y_test).flatten()

    var_name = target_col.replace("_obs", "")

    # 7. 예측 결과 데이터프레임 생성
    result_df = pd.DataFrame({
        "datetime": test_df["datetime"].values,
        f"{var_name}_pred": preds,
        f"{var_name}_true": trues
    })

    return result_df


In [15]:
# 각 변수별 모델 실행
df_temp = run_forecast_pipeline("temp_obs",     lr=0.0001, sched_patience=20, factor=0.3)
df_humi = run_forecast_pipeline("humidity_obs", lr=0.00001, sched_patience=20, factor=0.9)
df_pres = run_forecast_pipeline("pressure_obs", lr=0.00003, sched_patience=20, factor=0.4)

# datetime 기준 병합
merged_df = df_temp.merge(df_humi, on="datetime").merge(df_pres, on="datetime")

# sAError 계산
def compute_sAError(true, pred):
    rmse = np.sqrt(np.mean((true - pred) ** 2))
    mae = np.mean(np.abs(true - pred))
    return (rmse + mae) / 2
merged_df['pressure_true']/=1.33322
merged_df["pressure_pred"]/=1.33322
sa_temp = compute_sAError(merged_df["temp_true"], merged_df["temp_pred"])
sa_humi = compute_sAError(merged_df["humidity_true"], merged_df["humidity_pred"])
sa_pres = compute_sAError(merged_df["pressure_true"], merged_df["pressure_pred"])

print("\n📊 sAError 결과:")
print(f" - temp     : {sa_temp:.6f}")
print(f" - humidity : {sa_humi:.6f}")
print(f" - pressure : {sa_pres:.6f}")

# 총합 sAError (가중치 적용)
weights = [0.5, 0.3, 0.2]
total_sAError = weights[0]*sa_temp + weights[1]*sa_humi + weights[2]*sa_pres
print(f"\n✅ 총합 sAError = {total_sAError:.6f}")

# ✅ 최종 결과 저장
final_df = merged_df[["datetime", "humidity_pred", "temp_pred", "pressure_pred"]]
final_df.columns = ["datetime", "humidity", "temp", "pressure"]
final_df.to_csv("/content/drive/MyDrive/forecast_march_full.csv", index=False)
print("\n📁 forecast_march2025.csv 저장 완료!")



🚀 Target: temp_obs
[Epoch 0001] Train Loss: 0.713080 | Val sAError: 5.717346
[Epoch 0010] Train Loss: 0.110478 | Val sAError: 1.655318
[Epoch 0020] Train Loss: 0.101423 | Val sAError: 1.561035
[Epoch 0030] Train Loss: 0.096939 | Val sAError: 1.538366
[Epoch 0040] Train Loss: 0.093713 | Val sAError: 1.545891
[Epoch 0050] Train Loss: 0.091388 | Val sAError: 1.529574
[Epoch 0060] Train Loss: 0.089148 | Val sAError: 1.589146
[Epoch 0070] Train Loss: 0.087511 | Val sAError: 1.584650
[Epoch 0080] Train Loss: 0.086938 | Val sAError: 1.590476
[Epoch 0090] Train Loss: 0.086521 | Val sAError: 1.597244

🛑 Early stopping at epoch 92

🚀 Target: humidity_obs
[Epoch 0001] Train Loss: 0.899392 | Val sAError: 16.591209
[Epoch 0010] Train Loss: 0.616000 | Val sAError: 12.162769
[Epoch 0020] Train Loss: 0.464740 | Val sAError: 9.980891
[Epoch 0030] Train Loss: 0.436161 | Val sAError: 9.649940
[Epoch 0040] Train Loss: 0.424715 | Val sAError: 9.590842
[Epoch 0050] Train Loss: 0.417444 | Val sAError: 9.624